<a href="https://colab.research.google.com/github/abrahamjenie/Programming-for-Data-Analysis-CA1-Web-Scraping/blob/main/SalaryData_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Importing Packages
import requests
import json
import re
from bs4 import BeautifulSoup

In [11]:
url = 'https://www.salary.com/research/ie-salary/alternate/data-scientist-analytics-intermediate-salary/ie/dublin'
response=requests.get(url)
print(response) 
response.text

<Response [200]>


'\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n    <meta charset="utf-8" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n    <title>Data Scientist/Analytics , Intermediate Salary in Dublin, IE | Salary.com</title>\r\n    <meta name="description" content="The average salary of a Data Scientist/Analytics , Intermediate in Dublin, IE is €39,009 on March 19, 2021. View more salaries and where you could earn the most." />\r\n    <meta name="author" content="Site built by: Salary.com" />\r\n    <meta property="og:locale" content="en_US" />\r\n    <meta property="og:type" content="website" />\r\n    <meta property="og:title" content="Data Scientist/Analytics , Intermediate Salary in Dublin, IE | Salary.com" />\r\n    <meta property="og:description" content="The average salary of a Data Scientist/Analytics , Intermediate in Dublin, IE is €39,009 on March 19, 2021. View more salaries and where you could earn the most." />\r\n    <meta property="og:site_name"

In [12]:
soup = BeautifulSoup(response.text, 'html.parser')

In [14]:
pattern = re.compile(r'Occupation')
script = soup.find('script', {'type': 'application/ld+json'}, text=pattern)
script

<script type="application/ld+json">
    {
    "@context": "http://schema.org",
    "@type": "Occupation",
    "name": "Data Scientist/Analytics , Intermediate",
    "mainEntityOfPage": {
    "@type": "WebPage",
    "lastReviewed": "2021-03-19T00:00:00Z"
    },
    "description": "A Data Scientist is a professional who extensively works with Big Data in order to derive valuable business insights from it.
However, data scientists and predictive analytics professionals (PAPs) are more likely to hold an advanced degree.
Day-to-day activities of a Data Scientist sometimes can be predictable, and sometimes they are something out of the ordinary.
",
    "estimatedSalary": [{
    "@type": "MonetaryAmountDistribution",
    "name": "base",
    "currency": "EUR",
    "duration": "P1Y",
    "percentile25": "31559",
    "median": "39009",
    "percentile75": "46472"
    }]
        
            ,
            "occupationLocation": [{
            "@type": "City",
            "name": "Dublin, Ireland"


In [21]:
json_raw = script.contents[0]
json_raw

'\r\n    {\r\n    "@context": "http://schema.org",\r\n    "@type": "Occupation",\r\n    "name": "Data Scientist/Analytics , Intermediate",\r\n    "mainEntityOfPage": {\r\n    "@type": "WebPage",\r\n    "lastReviewed": "2021-03-19T00:00:00Z"\r\n    },\r\n    "description": "A Data Scientist is a professional who extensively works with Big Data in order to derive valuable business insights from it.\r\nHowever, data scientists and predictive analytics professionals (PAPs) are more likely to hold an advanced degree.\r\nDay-to-day activities of a Data Scientist sometimes can be predictable, and sometimes they are something out of the ordinary.\r\n",\r\n    "estimatedSalary": [{\r\n    "@type": "MonetaryAmountDistribution",\r\n    "name": "base",\r\n    "currency": "EUR",\r\n    "duration": "P1Y",\r\n    "percentile25": "31559",\r\n    "median": "39009",\r\n    "percentile75": "46472"\r\n    }]\r\n        \r\n            ,\r\n            "occupationLocation": [{\r\n            "@type": "City

In [22]:
json_data = json.loads(json_raw)

JSONDecodeError: ignored